In [1]:
import numpy as np 

## ニューラルネットワークの復習

In [6]:
w1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
x = np.random.randn(10, 2)

h = np.dot(x, w1) + b1

h.shape

(10, 4)

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
x = np.random.randn(10, 2)
w1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
w2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

h = np.dot(x, w1) + b1
a = sigmoid(h)
s = np.dot(a, w2) + b2
s.shape

(10, 3)

In [22]:
# layerの実装

# シグモイド
class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self, x):
        return 1 / (1 + np.exp(-x))


# 全結合
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]

    def forward(self, x):
        W, b = self.params
        out =  np.dot(x, W) + b
        return out

In [34]:
# aff -> sig -> aff

class TwolayerNet:
    def __init__(self, input_size, hidden_size, output_size):

        # レイヤーの生成
        I, H, O = input_size, hidden_size, output_size
        w1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        w2 = np.random.randn(H, O)
        b2 = np.random.randn(O)

        self.layers = [
            Affine(w1, b1),
            Sigmoid(),
            Affine(w2, b2)
        ]

        # パラメータをまとめる
        self.params = []
        for layer in self.layers:
            self.params += layer.params

    # 推論
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
        

In [35]:
# 実際に推論してみる
x = np.random.randn(10,2)
model = TwolayerNet(2, 4, 3)
s = model.predict(x)

In [40]:
model.params

[array([[-0.45169862, -0.22800835, -0.97227584, -0.37928813],
        [-2.17279999, -0.59518809,  1.96000837, -0.2470133 ]]),
 array([-0.59460211, -0.98188608,  0.62442778, -1.16866044]),
 array([[ 1.01374973, -0.37232415, -1.15848661],
        [ 0.16891674,  0.54420896,  0.79923485],
        [-1.88120017, -0.15914182,  0.83001465],
        [-0.55969501,  0.6535826 ,  1.08176436]]),
 array([-1.48414064,  0.07911855, -0.16433585])]

In [41]:
s

array([[-1.07863063,  0.14960528, -0.34004083],
       [-3.42803349,  0.09143038,  0.92831842],
       [-1.7830109 ,  0.13630769,  0.04829085],
       [-3.35649446,  0.07588756,  0.87139536],
       [-2.56264729,  0.14775557,  0.49238655],
       [-3.28752594,  0.17730357,  0.96369396],
       [-1.00903517,  0.13910992, -0.39132568],
       [-1.18459114,  0.12566364, -0.29887757],
       [-1.65496377,  0.15545846, -0.01455686],
       [-3.39700139,  0.16760898,  1.01331101]])